In [1]:
import pandas as pd
import json
from scipy.stats import ttest_rel
from scipy.stats import ttest_ind
from scipy.stats import fisher_exact
from scipy.stats import chi2_contingency
from scipy.special import rel_entr
import numpy as np
import networkx as nx
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_pickle("actor_movie_change.pkl")

In [3]:
df

,ActorName,ActorID,ActorDOB,ActorHeight,ActorGender,ActorAgeAtMovieRelease,Cannes,Berlin,Venice,OscarWinner,...,MovieGenresFinal,AverageRating,NumVotes,MovieBoxOfficeRevenue,InflationCoeff,InflationCorrectedRevenue,WindowValue,PeriodTag,PeriodChange,ChangeCounts
0,Adam Carolla,/m/010p3,1964,1.88,M,33.0,0,0,0,0,...,"[Comedy, Romance]",NaN,NaN,NaN,NaN,NaN,"[Comedy, Romance]",Comedy,0,0
1,Adam Carolla,/m/010p3,1964,1.88,M,34.0,0,0,0,0,...,[Comedy],NaN,NaN,45703.0,NaN,NaN,[Comedy],Comedy,0,0
2,Adam Carolla,/m/010p3,1964,1.88,M,35.0,0,0,0,0,...,"[Comedy, Romance]",5.00,15579.0,24419914.0,1.573576,3.842659e+07,[Comedy],Comedy,0,0
3,Adam Carolla,/m/010p3,1964,1.88,M,35.0,0,0,0,0,...,"[Comedy, Romance]",NaN,NaN,NaN,NaN,NaN,[Comedy],Comedy,0,0
4,Adam Carolla,/m/010p3,1964,1.88,M,38.0,0,0,0,0,...,[Comedy],NaN,NaN,NaN,NaN,NaN,[Comedy],Comedy,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202039,Paul Giamatti,/m/0zcbl,1967,1.74,M,44.0,0,0,0,0,...,"[Comedy, Detective, Fantasy, Horror, Thriller]",6.30,39846.0,NaN,1.180213,NaN,[Comedy],Comedy,0,11
202040,Paul Giamatti,/m/0zcbl,1967,1.74,M,44.0,0,0,0,0,...,[Drama],7.10,228252.0,75993061.0,1.204636,9.154395e+07,[Drama],Drama,1,11
202041,Paul Giamatti,/m/0zcbl,1967,1.74,M,44.0,0,0,0,0,...,[Drama],5.10,49512.0,6063556.0,1.180213,7.156288e+06,[Drama],Drama,0,11
202042,Paul Giamatti,/m/0zcbl,1967,1.74,M,45.0,0,0,0,0,...,"[Comedy, Musical, Romance]",7.05,80813.0,55518613.0,1.180213,6.552379e+07,[Drama],Drama,0,11


In [4]:
# Step 1: Calculating the proportion of valid values in InflationCorrectedRevenue and AverageRating for each actor
def calculate_valid_proportions(actor_data):
    valid_revenue_proportion = actor_data['InflationCorrectedRevenue'].notna().mean()
    valid_rating_proportion = actor_data['AverageRating'].notna().mean()
    return valid_revenue_proportion, valid_rating_proportion

valid_proportions_by_actor = df.groupby('ActorID').apply(calculate_valid_proportions)

# Step 2: Filtering actors with 50% or more valid values in both InflationCorrectedRevenue and AverageRating
actors_with_high_valid_proportions = valid_proportions_by_actor[(valid_proportions_by_actor.apply(lambda x: x[0] >= 0.5 and x[1] >= 0.5))]

actors_with_high_valid_proportions_index = actors_with_high_valid_proportions.index

# Filtering the dataset for these actors
actors_valid = df[df['ActorID'].isin(actors_with_high_valid_proportions_index)]